# Exporteer naar Excel met layout
Gebruik van Xslxwriter maakt export naar een Excel file met layout mogelijk.
Zie: https://xlsxwriter.readthedocs.io/working_with_pandas.html
Hier wordt een DECOR transactie geëxporteerd naar Excel met:
* Ieder topniveau uit de dataset op een apart tabblad
* Kolommen e.d. naar juiste breedte
* Valuesets ieder op een eigen tab

Eerst wat overhead: importeren van de nodige libraries.

In [1]:
import pandas as pd
import xlsxwriter
import json
import requests
import datetime
from IPython.display import JSON

## Ophalen van de DECOR transactie.
DECOR datasets en transacties zijn te vinden op: https://decor.nictiz.nl/decor/services/ProjectIndex

Kies een transactie met 'format=json'.

Na de GET moet status '200' getoond worden (afhankelijk van de gekozen transactie kan het even duren).

In [2]:
uri = 'https://decor.nictiz.nl/decor/services/RetrieveTransaction?id=2.16.840.1.113883.3.1937.99.62.3.4.5&effectiveDate=2012-09-05T16:59:35&language=nl-NL&ui=nl-NL&format=json'
resp = requests.get(uri)
resp.status_code

200

## Gegevens bekijken als json

In [3]:
data = resp.json()
dataset = data["dataset"][0]
name = dataset.get('shortName', 'sheet')
JSON(dataset)

<IPython.core.display.JSON object>

## Conversie naar DataFrame
De DECOR transaction wordt geconverteerd naar een pandas DataFrame.

In [4]:
def makeRows(parent, cols, valueSets, level=0, toplevel=None):
    level += 1
    frames = []
    for concept in parent["concept"]:
        if (level == 1):
            toplevel = concept['name'][0]['content']
        dict = {key: concept[key] for key in concept.keys() if key in cols}
        dict['name'] = concept['name'][0].get('content')
        dict['desc'] = concept['desc'][0].get('content')
        if 'valueDomain' in concept.keys():
            dict['datatype'] = concept['valueDomain'][0].get('type')
            dict['cc'] = concept.get('minimumMultiplicity', '') + '..' + concept.get('maximumMultiplicity', '') + ' ' + concept.get('conformance', '')
        if 'valueSet' in concept.keys():
            if ('name' in concept['valueSet'][0]):
                dict['valuesetDisplay'] = concept['valueSet'][0].get('displayName')
                dict['valuesetName'] = concept['valueSet'][0].get('name')
                valueSets[concept['valueSet'][0].get('name')] = concept['valueSet'][0]
        dict['level'] = level
        dict['toplevel'] = toplevel
        df = pd.DataFrame(dict, index=[dict['id']])
        frames.append(df)
        if 'concept' in concept.keys():
            frames.append(makeRows(concept, cols, valueSets, level, toplevel))
    return pd.concat(frames)

# cols = ['id', 'statusCode', 'effectiveDate', 'type', 'datatype', 'minimumMultiplicity', 'maximumMultiplicity', 'isMandatory', 'iddisplay', 'shortName']
cols = ['id', 'statusCode', 'type', 'datatype', 'minimumMultiplicity', 'maximumMultiplicity', 'conformance', 'isMandatory', 'iddisplay', 'shortName']
valueSets = {}
df = makeRows(dataset, cols, valueSets)
status = ['draft', 'pending', 'active']
df = df[df['statusCode'].isin(status)]
df

,id,statusCode,type,minimumMultiplicity,maximumMultiplicity,isMandatory,iddisplay,shortName,name,desc,level,toplevel,conformance,datatype,cc,valuesetDisplay,valuesetName
2.16.840.1.113883.3.1937.99.62.3.2.1,2.16.840.1.113883.3.1937.99.62.3.2.1,draft,group,0,*,false,demo1-dataelement-1,meetwaarden,Meetwaarden,"Metingen die op locatie bij de patiënt, veelal...",1,Meetwaarden,NaN,NaN,NaN,NaN,NaN
2.16.840.1.113883.3.1937.99.62.3.2.3,2.16.840.1.113883.3.1937.99.62.3.2.3,draft,item,1,1,true,demo1-dataelement-3,gewicht,Gewicht,Lichaamsgewicht,2,Meetwaarden,M,quantity,1..1 M,NaN,NaN
2.16.840.1.113883.3.1937.99.62.3.2.13,2.16.840.1.113883.3.1937.99.62.3.2.13,draft,item,0,1,false,demo1-dataelement-13,gewichtstoename,Gewichtstoename,Is er sprake van gewichtstoename?,2,Meetwaarden,NaN,boolean,0..1,NaN,NaN
2.16.840.1.113883.3.1937.99.62.3.2.18,2.16.840.1.113883.3.1937.99.62.3.2.18,draft,group,0,1,false,demo1-dataelement-18,gegevens_gewichtstoename,Gegevens gewichtstoename,Gegevens gewichtstoename,2,Meetwaarden,R,NaN,NaN,NaN,NaN
2.16.840.1.113883.3.1937.99.62.3.2.19,2.16.840.1.113883.3.1937.99.62.3.2.19,draft,item,0,1,false,demo1-dataelement-19,grootte_gewichtstoename,Grootte gewichtstoename,-,3,Meetwaarden,R,quantity,0..1 R,NaN,NaN
2.16.840.1.113883.3.1937.99.62.3.2.20,2.16.840.1.113883.3.1937.99.62.3.2.20,draft,item,0,1,false,demo1-dataelement-20,oorzaak_gewichtstoename,Oorzaak gewichtstoename,-,3,Meetwaarden,R,string,0..1 R,NaN,NaN
2.16.840.1.113883.3.1937.99.62.3.2.15,2.16.840.1.113883.3.1937.99.62.3.2.15,draft,item,0,1,false,demo1-dataelement-15,lengte,Lengte,Lichaamslengte,2,Meetwaarden,NaN,quantity,0..1,NaN,NaN
2.16.840.1.113883.3.1937.99.62.3.2.2,2.16.840.1.113883.3.1937.99.62.3.2.2,draft,item,0,1,false,demo1-dataelement-2,datum_en_tijd_meting,Datum en tijd meting,Datum meting lichaamsgewicht,2,Meetwaarden,NaN,datetime,0..1,NaN,NaN
2.16.840.1.113883.3.1937.99.62.3.2.5,2.16.840.1.113883.3.1937.99.62.3.2.5,draft,item,0,1,false,demo1-dataelement-5,meting_door,Meting door,Persoon die de meting verricht heeft,2,Meetwaarden,NaN,ordinal,0..1,vs-measured-by,vs-measured-by
2.16.840.1.113883.3.1937.99.62.3.2.22,2.16.840.1.113883.3.1937.99.62.3.2.22,draft,item,0,1,false,demo1-dataelement-22,beoordeling,Beoordeling,None,2,Meetwaarden,R,code,0..1 R,Weight Assesment,weight-assesment


## Opslaan als Excel
Het DataFrame exporteren naar Excel.

In [5]:
cols = ['name', 'iddisplay', 'statusCode', 'datatype', 'cc', 'desc', 'valuesetDisplay']
writer = pd.ExcelWriter('Example_02_' + name + '.xlsx')
workbook = writer.book
bold_format = workbook.add_format({'bold': True})
about = workbook.add_worksheet('About')
about.write(1, 1, 'Name')
about.write(1, 2, dataset['name'][0].get('content'))
about.write(2, 1, 'Desc')
about.write(2, 2, dataset['desc'][0].get('content'))
about.write(2, 1, 'Created')
about.write(2, 2, datetime.datetime.now().strftime('%c'))
about.set_column('B:B', 30, bold_format)
about.set_column('C:C', 30)
row = 3
for key in dataset.keys():
    if (key in ['name', 'desc', 'concept']):
        pass
    else:
        about.write(row, 1, key)
        about.write(row, 2, dataset[key])
        row += 1 
for sheetname in df['toplevel'].drop_duplicates():
    dfsheet = df[df["toplevel"] == sheetname]
    dfsheet[cols].to_excel(writer, index=False, sheet_name=sheetname)
    worksheet = writer.sheets[sheetname]
    worksheet.outline_settings(symbols_below=False)
    worksheet.freeze_panes(1, 1)
    worksheet.set_column('A:A', 30, bold_format)
    worksheet.set_column('B:B', 30)
    worksheet.set_column('C:C', 10)
    worksheet.set_column('F:F', 100)
    worksheet.set_column('G:G', 20)
    i = 0
    for level in dfsheet['level']:
        i += 1
        if (level > 1):
            worksheet.set_row(i, None, None, {'level': level - 1 if level - 1 < 7 else 7})  # max 7 group levels in Excel
for valueSet in sorted(valueSets):
    worksheet =  workbook.add_worksheet(valueSet[0:31])
    vs = valueSets[valueSet]
    worksheet.write(0, 0, 'Id')
    worksheet.write(0, 1, vs.get('id'))
    worksheet.write(1, 0, 'Name')
    worksheet.write(1, 1, vs.get('name'))
    worksheet.write(2, 0, 'DisplayName')
    worksheet.write(2, 1, vs.get('displayName'))
    worksheet.write(3, 0, 'EffectiveDate')
    worksheet.write(3, 1, vs.get('effectiveDate'))
    worksheet.write(4, 0, 'StatusCode')
    worksheet.write(4, 1, vs.get('statusCode'))
    worksheet.write(6, 0, 'Code')
    worksheet.write(6, 1, 'CodeSystem')
    worksheet.write(6, 2, 'DisplayName')
    worksheet.write(6, 3, 'Level')
    worksheet.write(6, 4, 'Type')
    worksheet.write(6, 5, 'Ordinal')
    worksheet.set_row(6, 18, bold_format)
    row = 7
    try:
        for concept in valueSets[valueSet]['conceptList'][0]['concept']:
            worksheet.write(row, 0, concept.get('code'))
            worksheet.write(row, 1, concept.get('codeSystem'))
            worksheet.write(row, 2, concept.get('displayName'))
            worksheet.write(row, 3, concept.get('level'))
            worksheet.write(row, 4, concept.get('type'))
            worksheet.write(row, 5, concept.get('ordinal'))
            worksheet.set_column('A:A', 30)
            worksheet.set_column('B:B', 60)
            worksheet.set_column('C:C', 60)
            row += 1
    except KeyError:
        pass
writer.save()